In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import pandas
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras


data_path = "../flowers/"
flower_types = os.listdir(data_path)
if '.DS_Store' in flower_types:
    flower_types.remove('.DS_Store')
print(flower_types)

data_labels =[]     #list containing all the labels such as daisy, rose..
data_set= []        #data set list conataining all images
image_names=[]      #list containing image file names
size= 100,100

for labels in flower_types:
    for file in os.listdir(os.path.join(data_path,labels)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data_path,labels,file))
            data_labels.append(labels)
            img = cv2.imread(os.path.join(data_path,labels,file))
            img_resize = cv2.resize(img,size)
            data_set.append(img_resize)
        else:
            continue

merged_dataset = list(zip(data_set, data_labels))  #Merging the images with its associated labels
random.shuffle(merged_dataset)                     #shuffling the data set
data_set, data_labels = zip(*merged_dataset)

#train_data, test_data, train_label, test_label = train_test_split(data_set, data_labels, test_size=0.3,random_state=9,stratify=data_labels)  #dividng the dataset to 70/30 train/test dataset
#train_data = np.array(train_data)
#train_label = np.array(train_label)
# test_data = np.array(test_data)
# test_label = np.array(test_label)

# label_dummies = pandas.get_dummies(train_label)
# train_labelidx =  label_dummies.values.argmax(1)
# #pandas.unique(train_labelidx)

# test_label_dummies = pandas.get_dummies(test_label)
# test_labelidx =  test_label_dummies.values.argmax(1)
#pandas.unique(test_labelidx)

train_data= np.array(data_set)
train_label = np.array(data_labels)
label_dummies = pandas.get_dummies(train_label)
train_labelidx =  label_dummies.values.argmax(1)

folds = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=9).split(train_data, train_labelidx))  #applying 10-fold cv
cvscores = []
cvloss=[]

for j,(train_idx,label_idx)  in enumerate(folds):
    
    print('\nFold ',j)
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(100,100,3)),
            keras.layers.Dense(128, activation=tf.nn.tanh),
            keras.layers.Dense(50, activation='relu'),
            keras.layers.Dense(5, activation=tf.nn.softmax)
        ])

    model.compile(optimizer=tf.train.AdamOptimizer(), 
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
                      #scoring="cross_val_score")

    History= model.fit(train_data[train_idx], train_labelidx[train_idx], epochs=10,validation_data=(train_data[label_idx], train_labelidx[label_idx]),verbose=1)

    score = model.evaluate(train_data[label_idx], train_labelidx[label_idx], batch_size=128,verbose=0)
    cvscores.append(score[1] * 100)  #accuracy
    cvloss.append(score[0])   #loss
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("%.2f%%" % (np.mean(cvloss)))

plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Sequential Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Sequential Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()





            




    







